# Exercise 05 ANN for Multi-class Classificatoin - Instruction

## Pedagogy

This notebook serves as an instruction for implementing ANNs using Pytorch to develop multi-class classification models.

Please use this notebook as a reference and guide to complete the assignment.

### Import libraries

In [ ]:
# import libraries
import warnings
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# suppress all warnings
warnings.filterwarnings("ignore")

## Part 1. ANN for Multi-class Classification

### Step 1. Build the data pipeline

We will use a toy dataset, the irir dataset, from `sklearn`. You can find more detials about this dataset at this [link](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris).

As the iris dataset is a tabular dataset that consists of numerical feature variables and a categorical target variable. We can use `torch.utils.data.TensorDataset`, a custom class that inherits from the `torch.utils.data.Dataset` class.

In this step, we need to:
1. Use `sklearn.datasets.load_iris()` to load the dataset
2. Perform feature scaling for the numerical feature variables
3. <span style="color:red">Perform one-hot encoding for the categorical target variable</span>
4. Divide the dataset into the training, validation, and test set
5. Create `TensorDataset` instances to store the training, validation, and test set
    - <span style="color:red">For numerical features, set `dtype = torch.float32`</span>
    - <span style="color:red">For categorical labels, set `dtype = torch.long`</span>
7. Create `DataLoader` instances to wrap the training, validation, and test set as iterable objects

In [ ]:
# load iris dataset
feature, label = datasets.load_iris(
    return_X_y = True,
    as_frame = False, # get data as numpy array
)

In [ ]:
# feature scaling
scaler = StandardScaler()
scaled_feature = scaler.fit_transform(feature)

In [ ]:
# one-hot encoding for label
encoder = OneHotEncoder(sparse = False)
encoded_label = encoder.fit_transform(label.reshape(-1, 1))
encoded_label

We can see the `encoded_label` has three columns, each corresponds to a class.
- If the instance belongs to the 1st class, then the value in the 1st columns is 1, and the values in the other columns are zero
- There can only be one `1` in a row, tha't why we call it one-hot
- Find more details in the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [ ]:
# Split the dataset into train, validation, and test sets
train_val_feature, test_feature, train_val_label, test_label = train_test_split(
    scaled_feature, label, test_size = 0.2, random_state = 0
)
train_feature, val_feature, train_label, val_label = train_test_split(
    train_val_feature, train_val_label, test_size = 0.25, random_state = 0
)

In [ ]:
# create the train, validatoin, and test dataset
# specify the type of data stored in the tensors to avoid incompatiblity
train_ds = TensorDataset(
    torch.tensor(train_feature, dtype = torch.float32),
    torch.tensor(train_label, dtype = torch.long)
)
val_ds = TensorDataset(
    torch.tensor(val_feature, dtype = torch.float32),
    torch.tensor(val_label, dtype = torch.long)
)
test_ds = TensorDataset(
    torch.tensor(test_feature, dtype = torch.float32),
    torch.tensor(test_label, dtype = torch.long)
)

For numerical featuers, we can set `dtype = torch.float32`.

For categorical labels, we need to set `dtype = torch.long`.

Find all available data types of `PyTorch` in the [documentation](https://pytorch.org/docs/stable/tensors.html).

In [ ]:
# create the train and test data loaders
batch_size = 8 # usually set to 2 to the nth power
train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
val_dl = DataLoader(val_ds, batch_size = batch_size, shuffle = False)
test_dl = DataLoader(test_ds, batch_size = batch_size, shuffle = False)

### Step 2. Create the artificial neural network

To define a neural network in `PyTorch`, we create a class that inherits from `torch.nn.Module`.

We create a multi-layer ANN here for the multi-class problem:
- We add <span style="color:red">batch normalization</span> layers and <span style="color:red">dropout</span> layers using:
    - `nn.BatchNorm1d()`
    - `nn.Dropout()`
- The number of output neurons is equal of the number of classes
- The activation functions of output neurons are linear
    - To get <span style="color:red">logits</span> as the outputs

In [ ]:
# define a custom neural network class
class MultiClassificationNet(nn.Module):
    def __init__(self, n_features, n_labels):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_features, 16),
            nn.BatchNorm1d(16), # batch normalization
            nn.ReLU(),
            nn.Dropout(p = 0.5), # dropout with 50% probability
            nn.Linear(16, 8),
            nn.BatchNorm1d(8), # batch normalization
            nn.ReLU(),
            nn.Dropout(p = 0.5), # dropout with 50% probability
            nn.Linear(8, n_labels)
        )
    def forward(self, X):
        return self.net(X)

In [ ]:
# create the neural network
model = MultiClassificationNet(
    n_features = feature.shape[1],
    n_labels = encoded_label.shape[1]
)

In [ ]:
# print network structure and learning parameters
print(f"Model structure:\n{model}\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()}")

We can see, apart from the linear layers, the batch normalization layers also have learning parameters.
- These learning parameters are also optimized during the training process
- The network may learn to deactivate the batch normalization, then:
    -  the learned weight is equal to the original standard deviation of the corresponding feature before batch normalization
    -  the learned bias is equal to the original mean of the corresponding feature before batch normalization
    -  This means we can let the ANN learn whether to use batch normalization or not by itself

### Step 3. Train the network

We define a `train()` function to perform the training process, which takes the following parameters:
- A data loader of the training set
- A data loader of the validation set
- A loss function (<span style="color:red">use `nn.CrossEntropyLoss()` for multi-class classification</span>)
- An optimizer (with L2 Regularization, learning rate decay, and early stopping)
- The number of epoches to train

There is one thing different for multi-class classification.
- <span style="color:red">The outputs of the ANN is logits</span>
    - not probabilities
    - not predictions
- <span style="color:red">The `nn.CrossEntropyLoss()` takes logits as inputs to compute the loss</span>
    - `nn.CrossEntropyLoss()` applies softmax internally

In [ ]:
# define the training function
def train(train_dl, val_dl, model, loss_fn, optimizer, epochs, early_stopping_patience, lr_scheduler):
    # initialization
    min_val_loss = np.inf # initialize the val loss as an infinite positive value
    patience_counter = 0 # set the initial patience counter to zero
    train_batch_loss_history = [] # for recording the average train loss of a batch
    train_epoch_loss_history = [] # for recording the average train loss of an epoch
    val_batch_loss_history = [] # for recording the average val loss of an batch
    val_epoch_loss_history = [] # for recording the average val loss of an epoch
    
    # start training
    for epoch in range(epochs): # iterate pre-defined number of epoches
        # train set
        train_epoch_loss = 0.0 # initial train epoch loss is set to zero
        model.train() # set the model in training mode
        for (X, y) in train_dl: # get a batch of training samples
            logits = model(X) # forward propagation
            train_batch_loss = loss_fn(logits, y) # compute the current train batch loss
            train_batch_loss.backward() # compute gradients by backpropagation
            optimizer.step() # update learning parameters according to gradients
            optimizer.zero_grad() # reset the gradients to zero
            train_batch_loss_history.append(train_batch_loss.item()) # record current train batch loss
            train_epoch_loss += train_batch_loss.item() # accumulate train batch losses for compute train epoch loss
        train_epoch_loss /= len(train_dl) # compute current train epoch loss
        train_epoch_loss_history.append(train_epoch_loss) # record current train epoch loss
        # validation set
        val_epoch_loss = 0.0 # initial val epoch loss is set to zero
        model.eval() # set the model in evaluation mode
        with torch.no_grad():
            for (X, y) in val_dl: # get a batch of validation samples
                logits = model(X) # forward propagation
                val_batch_loss = loss_fn(logits, y) # compute the current val batch loss
                val_batch_loss_history.append(val_batch_loss.item()) # record current val batch loss
                val_epoch_loss += val_batch_loss.item() # accumulate val batch losses for compute val epoch loss
            val_epoch_loss /= len(val_dl) # compute current val epoch loss
            val_epoch_loss_history.append(val_epoch_loss) # record current val epoch loss
        print(f"Epoch {epoch + 1}: train loss = {train_epoch_loss:>5f}, val loss = {val_epoch_loss:>5f}") # print log
        # learning rate decay
        old_lr = optimizer.param_groups[0]['lr']
        lr_scheduler.step(val_epoch_loss)
        new_lr = optimizer.param_groups[0]['lr']
        if old_lr != new_lr:
            print(f'Learning rate reduced after epoch {epoch+1}')
        # early stopping
        if val_epoch_loss < min_val_loss:
            min_val_loss = val_epoch_loss # update the new min val loss
            patience_counter = 0 # reset patience counter to zero
        else:
            patience_counter += 1 # increase patience counter by 1
        if patience_counter >= early_stopping_patience:
            print('Early stopping!')
            break
    return train_batch_loss_history, val_batch_loss_history, train_epoch_loss_history, val_epoch_loss_history

In [ ]:
# define the training hyper-parameters
loss_fn = nn.CrossEntropyLoss() # use cross entropy loss for multi-class
learning_rate = 1e-1
weight_decay = 1e-5 # lambda for L2 regularization
optimizer = torch.optim.SGD(
    model.parameters(),
    lr = learning_rate,
    weight_decay = weight_decay
)
epochs = 1000
early_stopping_patience = 20
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode = 'min',
    factor = 0.1, # each step multiply the learning rate with 0.1
    patience = 5
)

In [ ]:
# train the neural network
train_batch_loss_history, val_batch_loss_history, train_epoch_loss_history, val_epoch_loss_history = train(
    train_dl,
    val_dl,
    model,
    loss_fn,
    optimizer,
    epochs,
    early_stopping_patience,
    lr_scheduler
)

In [ ]:
# plot the train loss history
plt.figure()
batch = np.arange(len(train_batch_loss_history))
epoch = np.arange(len(train_epoch_loss_history))
batches_per_epoch = (int(len(train_feature) / batch_size + 1))
plt.plot(batch, train_batch_loss_history, '-', label = 'batch loss')
plt.plot(epoch * batches_per_epoch, train_epoch_loss_history, '-', label = 'epoch loss')
plt.title('train loss history')
plt.xlabel('batch')
plt.ylabel('train loss')
plt.legend()
plt.show()

In [ ]:
# plot the val loss history
plt.figure()
batch = np.arange(len(val_batch_loss_history))
epoch = np.arange(len(val_epoch_loss_history))
batches_per_epoch = (int(len(val_feature) / batch_size + 1))
plt.plot(batch, val_batch_loss_history, '-', label = 'batch loss')
plt.plot(epoch * batches_per_epoch, val_epoch_loss_history, '-', label = 'epoch loss')
plt.title('validation loss history')
plt.xlabel('batch')
plt.ylabel('val loss')
plt.legend()
plt.show()

In [ ]:
# plot the train vs. val loss history
plt.figure()
plt.plot(np.arange(len(train_epoch_loss_history)), train_epoch_loss_history, '-', label = 'train loss')
plt.plot(np.arange(len(val_epoch_loss_history)), val_epoch_loss_history, '-', label = 'val loss')
plt.title('train vs. validation loss history')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

### Step 4. Save and load the trained model

In [ ]:
# save model
file_name = 'multi_classification_net.pth'
torch.save(model.state_dict(), file_name)
print('Saved PyTorch Model State to '+ file_name)

In [ ]:
# load model
model = MultiClassificationNet(
    n_features = feature.shape[1],
    n_labels = encoded_label.shape[1]
)
model.load_state_dict(torch.load(file_name))

### Step 5. Make predictions and evaluation

Make predictions on the test set and evaluation the performance of the network.

As the outputs of the ANN are logits, which can be used to calculate the loss using `nn.CrossEntropyLoss()`, but <span style="color:red">we can't use these logits directly as the predicted probabilities or predicted classes</span>. We need to:
- <span style="color:red">Convert the logits to probabilities using `torch.softmax()`</span>
- <span style="color:red">Convert the probabilities to predicted classes using `torch.max()`</span>

In [ ]:
# define a function to make predictions on test dataset and evaluate the performance
def test(dataloader, model, loss_fn):
    batch_logits_list = [] # for recording batch logits
    batch_prob_list = [] # for recording batch probabilities
    batch_pred_list = [] # for recording batch predictions
    model.eval() # set the model in evaluation mode
    with torch.no_grad(): # disable automatic gradient computing
        loss = 0.0 # set initial test loss to zero
        for (X, y) in dataloader: # get a batch from test samples
            batch_logits = model(X) # forward propagation
            batch_prob = torch.softmax(batch_logits, dim = -1) # convert logtis to probabilities
            _, batch_pred = torch.max(batch_logits, 1) # conver probabilities to predictions
            batch_loss = loss_fn(batch_logits, y) # compute current batch loss
            loss += batch_loss.item() # accumulate batch losses for compute test loss
            batch_logits_list.append(batch_logits) # record logits on current batch
            batch_prob_list.append(batch_prob) # record probabilities on current batch
            batch_pred_list.append(batch_pred) # record predictions on current batch
        loss /= len(dataloader) # compute test loss
        logits = np.concatenate(batch_logits_list) # reform the logits
        prob = np.concatenate(batch_prob_list) # reform the probabilities
        pred = np.concatenate(batch_pred_list) # reform the predictions
        print(f"test loss = {loss}") # print log
    return logits, prob, pred, loss

In [ ]:
# make prediction on test set and evaluate the performance
test_logits, test_prob, test_pred, test_loss = test(test_dl, model, loss_fn)
print(classification_report(test_label, test_pred))

## Part 2. Obtain an over-fitted model

We have applied various techniques to prevent the model from overfitting:
- L2 regularization / weight decay
- early stopping
- learning rate decay
- Dropout
- Batch normalization

<span style="color:red">Now let's try to remove or deactivate them and increase the capacity of the ANN to see if you can get an over-fitted model.</span>

In practice, we always start with an over-fitted model to ensure it has enough capacity.

Then we adopt different techniques to try to prevent overfitting.